# Puzzle

https://thefiddler.substack.com/p/can-you-take-a-risk

**This Week’s Fiddler**

Earlier this week, I was playing the board game Risk with one of my kids. At the end of each turn in the game in which you conquer at least one enemy territory on the board, you are dealt a card.

There are 42 territory cards in the deck—14 that depict an infantry unit, 14 that depict a cavalry unit, and 14 that depict an artillery unit. Once you have three cards that either (1) all depict the same kind of unit, or (2) all depict different kinds of units, you can trade them in at the beginning of your next turn in exchange for some bonus units to be placed on the board.

If you are randomly dealt three cards from the 42, what is the probability that you can trade them in?

**This Week’s Extra Credit**

The full deck of Risk cards also contains two wildcards, which can be used as any of the three types of cards (infantry, cavalry, and artillery) upon trading them in. Thus, the full deck consists of 44 cards.

You must have at least three cards to have any shot at trading them in. Meanwhile, having five cards guarantees that you have three you can trade in.

If you are randomly dealt cards from a complete deck of 44 one at a time, how many cards would you need, on average, until you can trade in three? (Your answer should be somewhere between three and five. And no, it’s not four.)

# Fiddler Solution

This seems like a good problem for a direct counting approach, as long as we are careful about our counting. :)

The total number of ways to draw 3 cards is 42 * 41 * 40

For all to be the same, we pick one of the 3 groups, and pick 3 cards from that group. This can be done in 3 * 14 * 13 * 12 ways.

For all different, we pick an ordering of groups, and then pick one card from each group. This can be done in 6 * 14 * 14 * 14 ways.

The probability can be calculated by adding and taking the ratio, simplifying by cancelling 42*4 throughout, etc.

This yields 137/410, which is close to 1/3, but not quite.

> An alternative approach is to use probabilities. 
> For all to be the same,  the first card can be anything (p = 42/42 = 1), the second card must be the same as the first (p = 13/41), and the third must also be the same (p = 12/40).
> For all different,  the first card can be anything (p = 42/42 = 1), the second card must be the different from the first (p = 28/41), and the third must be different from the first 2 (p = 14/40). This should also yield the same result. 

In [30]:
from fractions import Fraction as F
from itertools import permutations

Fiddler_answer_counting = F((3 * 14 * 13 * 12 + 6 * 14 * 14 * 14), (42 * 41 * 40))

Fiddler_answer_probabilities = F(42,42) * F(13,41) * F(12,40) + F(42,42) * F(28,41) * F(14,40) 
assert Fiddler_answer_counting == Fiddler_answer_probabilities

deck_without_wildcards = "A" * 14 + "I" * 14 + "C" * 14
Hand_count, E_sum = 0, 0
for hand in permutations(deck_without_wildcards, 3):
    Hand_count += 1
    if hand[0] == hand[1] == hand[2] or (hand[0] != hand[1] and hand[1] != hand[2] and hand[0] != hand[2]):
        E_sum += 1
Fiddler_answer_exhaustive = F(E_sum, Hand_count)
assert Fiddler_answer_exhaustive == Fiddler_answer_counting

print (f"Fiddler answer: {Fiddler_answer_counting} = {float(Fiddler_answer_counting):.6f}")

Fiddler answer: 137/410 = 0.334146


# Extra Credit Solution.

In any instance of cards being dealt, the number of cards you are dealt before you can trade in can either be 3, 4 or 5. Figuring out the probabilities for these 3 events should make the answer easy to find.

P3 can be calculated in a way similiar to the Fiddler, except that there are 2 more cases to consider, 1 wildcard or 2 wildcards. In either case, you are sure to be able to trade in because X,X,wildcard and X,Y,wildcard can be treated as either all same or all different.

P5 is also easy to calculate. The only way to reach 5 cards is if your first 4 cards fall into just 2 categories. There are 3 ways to pick 2 categories and 4!/(2! * 2!) = 6 ways to arrange them, and then 14 * 13 ways to pick card for each category.

P4 = 1 - P4 -P5

And the desired expected value is 3 * P3 + 4 * P4 + 5 * P5.

This can be simplified to 4 - P3 + P5.

In [31]:
# github copiliot guiding me to better variable names than I would have chosen myself :)
Ways_to_choose_3_cards = 44 * 43 * 42
Ways_to_choose_all_same = 3 * 14 * 13 * 12
Ways_to_choose_all_different = 6 * 14 * 14 * 14
Ways_to_choose_one_wildcard = 3 * 42 * 41 * 2 
Ways_to_choose_two_wildcards = 3 * 42 * 2 * 1

P3 = F(Ways_to_choose_all_same + Ways_to_choose_all_different + Ways_to_choose_one_wildcard + Ways_to_choose_two_wildcards, Ways_to_choose_3_cards)

Ways_to_choose_4_cards = 44 * 43 * 42 * 41
Ways_to_choose_4_in_2_groups_of_2 = 3 * 6 * 14 * 13 * 14 * 13

P5 = F(Ways_to_choose_4_in_2_groups_of_2, Ways_to_choose_4_cards)

P4 = 1 - P3 - P5

print(f"P3 = {P3} = {float(P3):.6f}")
print(f"P4 = {P4} = {float(P4):.6f}")
print(f"P5 = {P5} = {float(P5):.6f}")

Expected_cards_explicit = 3 * P3 + 4 * P4 + 5 * P5

Expected_cards_simplified = 4 - P3 + P5
assert Expected_cards_explicit == Expected_cards_simplified

print(f"Expected cards drawn = {Expected_cards_explicit} = {float(Expected_cards_explicit):.6f}")

P3 = 200/473 = 0.422833
P4 = 7644/19393 = 0.394163
P5 = 3549/19393 = 0.183004
Expected cards drawn = 72921/19393 = 3.760171


In [34]:
from functools import cache
@cache
def num_cards_from_hand(hand):
    first_3_cards = hand[0:3]
    first_4_cards = hand[0:4]
    if first_3_cards[0] == first_3_cards[1] == first_3_cards[2]:
        return 3
    elif first_3_cards[0] != first_3_cards[1] and first_3_cards[1] != first_3_cards[2] and first_3_cards[0] != first_3_cards[2]:
        return 3
    elif first_3_cards.count('W') >= 1:
        return 3
    elif first_4_cards[3] == 'W':
        return 4
    elif first_4_cards.count('A') >= 3 or first_4_cards.count('I') >= 3 or first_4_cards.count('C') >= 3:
        return 4
    elif first_4_cards.count('A') >= 1 and first_4_cards.count('I') >= 1 and first_4_cards.count('C') >= 1:
        return 4
    else:
        return 5

assert num_cards_from_hand("AAAIW") == 3
assert num_cards_from_hand("AIWAI") == 3
assert num_cards_from_hand("AICAI") == 3
assert num_cards_from_hand("AIICC") == 4
assert num_cards_from_hand("AIAIC") == 5

deck_with_wildcards = deck_without_wildcards + "W" * 2
Hand_count, E_sum, P3_cnt, P4_cnt, P5_cnt = 0, 0, 0, 0, 0
for hand in permutations(deck_with_wildcards, 5):
    Hand_count += 1
    nch = num_cards_from_hand(hand)
    E_sum += nch
    if nch == 3:
        P3_cnt += 1
    elif nch == 4:
        P4_cnt += 1
    elif nch == 5:
        P5_cnt += 1
    else:
        raise ValueError("Unexpected number of cards")

P3_exhaustive = F(P3_cnt, Hand_count)
P4_exhaustive = F(P4_cnt, Hand_count)
P5_exhaustive = F(P5_cnt, Hand_count)
Expected_cards_exhaustive = F(E_sum, Hand_count)
print(f"P3 (exhaustive) = {P3_exhaustive} = {float(P3_exhaustive):.6f}")
print(f"P4 (exhaustive) = {P4_exhaustive} = {float(P4_exhaustive):.6f}")
print(f"P5 (exhaustive) = {P5_exhaustive} = {float(P5_exhaustive):.6f}")
print(f"Expected cards drawn (exhaustive) = {Expected_cards_exhaustive} = {float(Expected_cards_exhaustive):.6f}")
assert P3_exhaustive == P3
assert P4_exhaustive == P4
assert P5_exhaustive == P5
assert Expected_cards_exhaustive == Expected_cards_explicit

P3 (exhaustive) = 200/473 = 0.422833
P4 (exhaustive) = 7644/19393 = 0.394163
P5 (exhaustive) = 3549/19393 = 0.183004
Expected cards drawn (exhaustive) = 72921/19393 = 3.760171
